# RNN을 이용한 텍스트 생성
- '경마장에 있는 말이 뛰고 있다'
- '그의 말이 법이다'
- '가는 말이 고와야 오는 말이 곱다'

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
seed= 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
text=""" 경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n 
가는 말이 고와야 오는 말이 곱다\n"""

In [5]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [6]:
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [7]:
# 단어 집합 크기 설정
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야함 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [9]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [11]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [12]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [13]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len = max(len(s) for s in sequences) 
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 6


In [14]:
# 전체 샘플의 길이를 6(가장 긴 샘플의 길이)으로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [15]:
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

In [16]:
# 각 샘플의 마지막 단어를 레이블로 분리
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [17]:
X[:3]

array([[0, 0, 0, 0, 2],
       [0, 0, 0, 2, 3],
       [0, 0, 2, 3, 1]], dtype=int32)

In [18]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
Y = to_categorical(y, num_classes=vocab_size)

In [19]:
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

### 모델 정의
- Embedding
- SimpleRNN

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 4)              48        
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 32)                1184      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                396       
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [30]:
history = model.fit(X, Y, epochs=200, verbose=0)

In [31]:
history.history['accuracy'][-1]

0.9090909361839294

### 모델 검증

In [32]:
# 모델 검증용 문장을 생성하는 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        #result = model.predict_classes(encoded, verbose=0)
        result = np.argmax(model.predict(encoded), axis=-1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [33]:
sentence_generation(model, t, '경마장에', 3)

'경마장에 말이 말이 뛰고'

In [34]:
sentence_generation(model, t, '그의', 2)

'그의 말이 법이다'

In [35]:
sentence_generation(model, t, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

- sentence_generation() 함수의 풀이 과정

In [36]:
enc = t.texts_to_sequences(['경마장에 말이 고와야'])[0]
enc = pad_sequences([enc], maxlen=5, padding='pre')
enc

array([[0, 0, 2, 1, 9]], dtype=int32)

In [39]:
result = np.argmax(model.predict(enc), axis=-1)
result

array([4])

In [38]:
for word, index in t.word_index.items(): 
    if index == result: 
        break 
print(word)

뛰고


### 모델변화
- Embedding Vector 갯수:[2, 4, 6, 8]
- RNN 노드 갯수:[24, 32]

In [40]:
def execute_model(n_embed, n_rnn):
    model = Sequential()
    model.add(Embedding(vocab_size, n_embed, input_length=max_len-1))
    model.add(SimpleRNN(n_rnn))
    model.add(Dense(vocab_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', metrics=['accuracy'])
    history = model.fit(X, Y, epochs=200, verbose=0)
    print(f"Accuracy: {history.history['accuracy'][-1]:.4f}")

    print(sentence_generation(model, t, '경마장에', 3))
    print(sentence_generation(model, t, '그의', 2))
    print(sentence_generation(model, t, '가는', 5))

In [41]:
for n_embed in [2,4,6,8]:
    for n_rnn in [24,32]:
        print('===================================')
        print(f'n_embed={n_embed}, n_rnn={n_rnn}')
        execute_model(n_embed, n_rnn)
        print()

n_embed=2, n_rnn=24
Accuracy: 0.8182
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

n_embed=2, n_rnn=32
Accuracy: 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=4, n_rnn=24
Accuracy: 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=4, n_rnn=32
Accuracy: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=6, n_rnn=24
Accuracy: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=6, n_rnn=32
Accuracy: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=8, n_rnn=24
Accuracy: 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=8, n_rnn=32
Accuracy: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

